In [ ]:
# -*- coding: UTF-8 -*-
#2016-02-28
import MySQLdb
import numpy as np
import random
from collections import Counter
import scipy
from pandas import *
from sklearn.ensemble import RandomForestClassifier
import time


In [ ]:

# ---------------------------------------------------------------------------------------------------------------------
SENSORDIRS = {'accelerometer': r"HASC1001-acc.csv"}
AXES = (('x', 3), ('y', 2), ('z', 1))
#the features we use
SINGLE_FEATURES = {
    'average-absolute-difference': 1,'standarddev': np.std, 'variance': np.var,
    'nanmin':np.nanmin, 'nanmax':np.nanmax, 'median':np.median, 'average':np.average,
    'percentile_70':2,'percentile_80':3,'percentile_90':4,
    'dominant-frequency': 5,'energy': 6
} #all features working on one axis
AXIS_TUPLE_NAME = 0
ACTIVITY_LABELS = ['stDown','stUp','walk','jog','stay','skip']

def generateHeader():
    '''
        Builds the header for a .arff file (WEKA file format)
    '''

    header = ['@RELATION activity']
    header.append('\n')
    sensor = SENSORDIRS.keys()[0]

    for fKey, func in SINGLE_FEATURES.items():
        for axis in AXES:
            header.append(''.join(['@ATTRIBUTE ', sensor, '-', axis[AXIS_TUPLE_NAME], '-', fKey, ' NUMERIC']))

    header.append(''.join(['@ATTRIBUTE class {', ', '.join(ACTIVITY_LABELS), '}']))
    header.append('\n')
    header.append('@DATA')
    # for i in  header:
    #     print i
    return header



def conn():
    # Open a database connection.
    db = MySQLdb.connect('localhost','root','meng','AccelerationData' )

    with db:
        #Still, the first step is to obtain the cursor object for the connection, which is used to execute queries.
        cur = db.cursor()
        sql1 = "set interactive_timeout = 2880000"
        cur.execute(sql1)
        sql2 = "set wait_timeout=2880000"
        cur.execute(sql2)
        #Similar to the query function in other languages, execute is the function for executing queries in Python.


        #Separate queries
        # args=['Person1201', 'Person1202']
        # rates=[100,75]
        # for i in range(len(args)):
        #     for j in range(len(rates)):
        #         sql = "SELECT * FROM AccelerationData.featuredata WHERE PersonID ='" + str(args[i]) + "'AND SampleRate ='" + str(rates[j])+"'"
        #         cur.execute(sql)
        #

        args=()
        allperson=range(1201,1308)
        # choose=random.sample(allperson,10)

        #10-1
        # choose=[1259, 1291, 1294, 1209, 1241, 1231, 1264, 1275, 1215, 1208]
        #10-2
        # choose=[1293, 1246, 1208, 1254, 1215, 1225, 1264, 1219, 1291, 1233]


        #30-1
        # choose=[1205, 1241, 1297, 1223, 1296, 1214, 1304, 1277, 1207, 1246, 1221, 1239, 1266, 1238, 1249, 1281, 1228, 1262, 1218, 1211, 1299, 1244, 1279, 1206, 1301, 1307, 1291, 1254, 1233, 1253]
        #30-2
        # choose=[1224, 1244, 1249, 1225, 1296, 1247, 1291, 1234, 1229, 1201, 1271, 1203, 1283, 1246, 1304, 1267, 1264, 1277, 1254, 1281, 1301, 1288, 1270, 1279, 1285, 1262, 1295, 1222, 1221, 1306]

        #50
        # choose=[1240, 1265, 1234, 1302, 1291, 1271, 1233, 1225, 1270, 1298, 1245, 1263, 1222, 1259, 1266, 1273, 1281, 1215, 1274, 1226, 1247, 1246, 1241, 1248, 1256, 1306, 1280, 1230, 1242, 1211, 1219, 1276, 1282, 1220, 1277, 1253, 1269, 1260, 1264, 1237, 1294, 1258, 1304, 1217, 1212, 1210, 1252, 1257, 1243, 1289]

        #70
        # choose=[1275, 1227, 1271, 1272, 1273, 1225, 1201, 1248, 1282, 1253, 1251, 1301, 1212, 1210, 1286, 1229, 1295, 1243, 1269, 1224, 1203, 1226, 1279, 1290, 1257, 1287, 1207, 1244, 1232, 1297, 1270, 1254, 1302, 1208, 1217, 1298, 1215, 1261, 1299, 1214, 1206, 1236, 1256, 1220, 1284, 1277, 1202, 1250, 1274, 1205, 1307, 1259, 1209, 1304, 1289, 1288, 1291, 1255, 1239, 1264, 1242, 1300, 1241, 1222, 1278, 1293, 1296, 1238, 1268, 1230]
        #100
        choose=[1256, 1307, 1268, 1266, 1291, 1300, 1210, 1232, 1302, 1276, 1216, 1293, 1285, 1261, 1284, 1278, 1292, 1248, 1280, 1251, 1267, 1206, 1304, 1212, 1258, 1207, 1287, 1221, 1238, 1271, 1243, 1272, 1214, 1299, 1255, 1265, 1260, 1203, 1263, 1259, 1294, 1296, 1253, 1233, 1242, 1305, 1247, 1306, 1262, 1211, 1209, 1295, 1281, 1246, 1236, 1235, 1230, 1204, 1219, 1289, 1301, 1224, 1254, 1244, 1257, 1208, 1264, 1277, 1213, 1288, 1270, 1202, 1240, 1274, 1282, 1241, 1217, 1205, 1297, 1252, 1229, 1298, 1303, 1249, 1218, 1201, 1223, 1250, 1215, 1226, 1273, 1279, 1234, 1283, 1275, 1290, 1225, 1286, 1231, 1222]
        # sample_rate=5
        samplelist=[100,95,90,85,80,75,70,65,60,55,50,45,40,35,30,25,20,15,10,5,4]
        print choose
        for sample_rate in samplelist:
            for c in  choose:
                args = args+('Person'+str(c),)
            # args=('Person1201', 'Person1202')
            rates=(sample_rate)
            sql_command ="SELECT * FROM AccelerationData.featuredata_new where PersonID IN "+str(args)+ "and SampleRate ="+ str(rates)
            cur.execute(sql_command)
            #Using the fetchall function, store the result set (a multidimensional tuple) into rows.
            rows = cur.fetchall()
            data = np.asarray(rows)
            actionlist=data[:,2]
            # personlist=data[:,1]
            # personlabel=unique(personlist)
            data = scipy.delete(data, (0,1,2,3,4,5,6), 1)
            sql = "set interactive_timeout=24*3600"
            cur.execute(sql)
            header=generateHeader()
            addlabel=np.column_stack((data,actionlist))
            for i in  addlabel.tolist():
                header.append(i)

            with open('/media/soumia/files/thesis/2018-01-22/Recognition/Action Recognition/'+str(len(choose))+'_'+str(sample_rate)+'_activity.arff', 'a') as the_file:
                for i in header:
                    if '@' not in i:
                        the_file.write(str(i)[1:-1].replace('\'','')+'\n')
                    else:
                        the_file.write(i+'\n')
        db.commit()
        # Close the database connection
        # db.close()



In [ ]:
def main():
    conn()

if __name__ == '__main__':
    begin = time.clock()
    main()
    end = time.clock()
    print end-begin